In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

# <font color='orange'>01. load data

In [2]:
rootpath = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\temp'

In [3]:
os.listdir(rootpath)

['temp1_inner_total_df.csv',
 'temp2_inner_total_df.csv',
 'temp3_inner_total_df.csv',
 'temp4_inner_total_df.csv',
 'temp5_inner_total_df.csv',
 'total_df.pkl']

In [4]:
total_df = pd.read_pickle(os.path.join(rootpath, 'total_df.pkl'), )

In [5]:
inner_total_df = pd.read_csv(os.path.join(rootpath, 'temp5_inner_total_df.csv'), )

In [6]:
total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,AS0000136,AS0000137,AS0000138,AS0000139,...,P2O000500_1_s6,LA0000001_1_s9,LA0000020_1_s9,LA0000227_1_s9,P2E000500_1_s9,LA0000001_1_s12,LA0000020_1_s12,LA0000204_1_s12,LA0000227_1_s12,P2O000500_1_s12
0,1,201612,NaN,5.0,5.0,0.0,26,-1,1,0,...,0.0,0.15,0.15,345.45,0.0,0.09,0.09,214.29,214.29,0.0
1,2,201612,D,5.0,2.0,0.0,-1,32,1,0,...,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0


In [7]:
inner_total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,INCOME,INCOME_INTERVAL,UDIR,UDIR_INTERVAL,SEG,FINAL_SEG,CAP_34,THE_DEAD,NOT_DEV_DATE_FG,DEV_TARGET_FG
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,30000,7,65.92,2,2,1,0,0,0,1
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,19000,3,68.25,2,1,1,0,0,0,1


# <font color='orange'>02. 현행 VER7 부도 정의 준용

### 채무불이행/공공정보등록

In [9]:
def _get_bad_common(TPERF0003, TPERF0005):
    bad_common = 0
    
    if TPERF0003 > 0 or TPERF0005 > 0:
        bad_common = 1
    else:
        bad_common = 0
        
    return bad_common


In [10]:
bad_need_cols = ['TPERF0003', 'TPERF0005',]

In [11]:
total_df['BAD_공공'] = total_df[bad_need_cols].apply(lambda x: _get_bad_common(x['TPERF0003'], x['TPERF0005'],), axis = 1)

In [12]:
inner_total_df['BAD_공공'] = total_df['BAD_공공']

### 금융연체 1년 60일 이상

In [13]:
def _get_bad_finan(TPERF0013, DQ_BANK_12, DQ_LEASE_12, DQ_CARD_12,
                  DQ_CAPITAL_12, DQ_SB_12,):
    bad_finan = 0
    
    if (TPERF0013 >= 60 or  DQ_BANK_12 >= 60 or  DQ_LEASE_12 >= 60 or  
        DQ_CARD_12 >= 30 or DQ_CAPITAL_12 >= 30 or  DQ_SB_12 >= 30):
        bad_finan = 1
    else:
        bad_finan = 0
        
    return bad_finan

In [14]:
bad_need_cols = ['TPERF0013', 'DQ_BANK_12', 'DQ_LEASE_12', 'DQ_CARD_12',
                  'DQ_CAPITAL_12', 'DQ_SB_12']

In [15]:
total_df['BAD_금융_12'] = total_df[bad_need_cols].apply(lambda x: _get_bad_finan(x['TPERF0013'],
                                                                                   x['DQ_BANK_12'],
                                                                                    x['DQ_LEASE_12'],
                                                                                    x['DQ_CARD_12'],
                                                                                    x['DQ_CAPITAL_12'],
                                                                                    x['DQ_SB_12'],
                                                                                   ), axis = 1)

In [16]:
inner_total_df['BAD_금융_12'] = total_df['BAD_금융_12']

#### 업권별 금융연체 1년내 60/30일 이상

In [17]:
def _get_bad_busi(TPERF0013, DQ_BANK_12, DQ_LEASE_12, DQ_CARD_12,
                  DQ_CAPITAL_12, DQ_SB_12):
    bad_short = 0
    bad_bank = 0
    bad_lease = 0
    bad_card = 0
    bad_capital = 0
    bad_sb = 0
    
    if TPERF0013 >= 60:
        bad_short = 1

    if DQ_BANK_12 >= 60:
        bad_bank = 1
 
    if DQ_LEASE_12 >= 60:
        bad_card = 1

    if DQ_CARD_12 >= 30:
        bad_lease = 1
   
    if DQ_CAPITAL_12 >= 30:
        bad_capital = 1
  
    if DQ_SB_12 >= 30:
        bad_sb = 1
    
    return bad_short, bad_bank, bad_lease, bad_card, bad_capital, bad_sb
        

In [18]:
bad_need_cols = ['TPERF0013', 'DQ_BANK_12', 'DQ_LEASE_12', 'DQ_CARD_12',
                  'DQ_CAPITAL_12', 'DQ_SB_12']

In [19]:
result_cols = ['BAD_금융_단기_12','BAD_금융_은행_12','BAD_금융_리스_12',
              'BAD_금융_카드_12','BAD_금융_캐피탈_12','BAD_금융_저축은행_12',]

In [20]:
total_df[result_cols] = total_df[bad_need_cols].apply(lambda x: _get_bad_busi(x['TPERF0013'],
                                                                           x['DQ_BANK_12'],
                                                                            x['DQ_LEASE_12'],
                                                                            x['DQ_CARD_12'],
                                                                            x['DQ_CAPITAL_12'],
                                                                            x['DQ_SB_12'],
                                                                           ), axis = 1, result_type = 'expand')

In [21]:
inner_total_df[result_cols] = total_df[result_cols]

### 대부연체 1년내 30일 이상

In [22]:
def _get_bad_ml(DQ_DB_12):
    bad_ml = 0
    
    if DQ_DB_12 >= 30:
        bad_ml = 1
        
    return bad_ml

In [23]:
total_df['BAD_대부_12'] = total_df['DQ_DB_12'].apply(lambda x: _get_bad_ml(x))

In [24]:
inner_total_df['BAD_대부_12'] = total_df['BAD_대부_12']

### 금융연체 6개월내 10일 초과

In [25]:
def _get_bad_finan(DQ_CARD_6, 
                  DQ_CAPITAL_6, DQ_SB_6,):
    bad_finan = 0
    
    if (DQ_CARD_6 > 10 or 
        DQ_CAPITAL_6 > 10 or  DQ_SB_6 > 10):
        bad_finan = 1
    else:
        bad_finan = 0
        
    return bad_finan

In [26]:
bad_need_cols = ['DQ_CARD_6',
                  'DQ_CAPITAL_6', 'DQ_SB_6']

In [27]:
total_df['BAD_금융_6'] = total_df[bad_need_cols].apply(lambda x: _get_bad_finan(
                                                                    x['DQ_CARD_6'],
                                                                    x['DQ_CAPITAL_6'],
                                                                    x['DQ_SB_6'],
                                                                   ), axis = 1)

In [28]:
inner_total_df['BAD_금융_6'] = total_df['BAD_금융_6']

#### 업권별 금융연체 6개월내 10일 초과

In [29]:
def _get_bad_busi(DQ_CARD_6, 
                  DQ_CAPITAL_6, DQ_SB_6):
    bad_card = 0
    bad_capital = 0
    bad_sb = 0
    

 
    if DQ_CARD_6 > 10:
        bad_card = 1

   
    if DQ_CAPITAL_6 > 10:
        bad_capital = 1
  
    if DQ_SB_6 > 10:
        bad_sb = 1
    
    return bad_card,  bad_capital, bad_sb
        

In [30]:
bad_need_cols = [ 'DQ_CARD_6',
                  'DQ_CAPITAL_6', 'DQ_SB_6']

In [31]:
result_cols = ['BAD_금융_카드_6',
            'BAD_금융_캐피탈_6','BAD_금융_저축은행_6',]

In [32]:
total_df[result_cols] = total_df[bad_need_cols].apply(lambda x: _get_bad_busi(
                                                                                x['DQ_CARD_6'],
                                                                                x['DQ_CAPITAL_6'],
                                                                                x['DQ_SB_6'],
                                                                               ), axis = 1, result_type = 'expand')

In [33]:
inner_total_df[result_cols] = total_df[result_cols]

### 대부연체 6개월내 10일 초과

In [34]:
def _get_bad_ml(DQ_DB_6):
    bad_ml = 0
    
    if DQ_DB_6 > 10:
        bad_ml = 1
        
    return bad_ml

In [35]:
total_df['BAD_대부_6'] = total_df['DQ_DB_6'].apply(lambda x: _get_bad_ml(x))

In [36]:
inner_total_df['BAD_대부_6'] = total_df['BAD_대부_6'] 

### 최종 BAD

In [37]:
def _get_bad(bad_common, bad_finan_12,bad_ml_12,
             bad_finan_6, bad_ml_6):
    bad = 0
    
    if (bad_common > 0 or bad_finan_12 > 0 or bad_ml_12 > 0 or
       bad_finan_6 > 0 or bad_ml_6 > 0) :
        bad = 1
    
    return bad

In [38]:
bad_need_cols = ['BAD_공공', 'BAD_금융_12', 'BAD_대부_12',
                 'BAD_금융_6','BAD_대부_6',]

In [39]:
total_df['BAD'] = total_df[bad_need_cols].apply(lambda x: _get_bad(x['BAD_공공'],
                                                                   x['BAD_금융_12'],
                                                                   x['BAD_대부_12'],
                                                                   x['BAD_금융_6'],
                                                                   x['BAD_대부_6']), axis = 1)

In [40]:
inner_total_df['BAD'] = total_df['BAD']

In [41]:
inner_total_df.head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,BAD_금융_카드_12,BAD_금융_캐피탈_12,BAD_금융_저축은행_12,BAD_대부_12,BAD_금융_6,BAD_금융_카드_6,BAD_금융_캐피탈_6,BAD_금융_저축은행_6,BAD_대부_6,BAD
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# <font color='orange'> 03. 저장

In [42]:
inner_total_df.to_csv(os.path.join(rootpath, 'final_inner_total_df.csv'), index = False)